This notebook includes the official implementation of the paper [**Anime Recommendation System Using Bert and Cosine Similarity**](https://doi.org/10.1109/AiDAS60501.2023.10284693) (AiDAS 2023).

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
from tabulate import tabulate

In [2]:
# Import Data
data = pd.read_csv('/kaggle/input/anime-recommendation-database-2020/anime.csv')
data = data[['MAL_ID','Name', 'Genres', 'Type', 'Episodes']]
data.columns = ['ID','Title', 'Genre', 'Type', 'Episodes']
len(data)

17562

In [3]:
# Data Cleaning
# Remove Missing Value Data
print(data.isnull().sum())
data = data.dropna()
print(data.isnull().sum())
print(len(data))

ID          0
Title       0
Genre       0
Type        0
Episodes    0
dtype: int64
ID          0
Title       0
Genre       0
Type        0
Episodes    0
dtype: int64
17562


In [4]:
# Data Preprocessing
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

data["cleaned_Title"] = data["Title"].apply(clean)
data["cleaned_Genre"] = data["Genre"].apply(clean)
data["cleaned_TitleGenre"] = data["cleaned_Title"] + " " + data["cleaned_Genre"]

In [5]:
indices = pd.Series(data.index,index=data['cleaned_TitleGenre']).drop_duplicates()

In [6]:
# Generate Similarity
def get_similarity(title):
    new_title = False
    feature = data["cleaned_TitleGenre"].tolist()
    if not(title in feature):
        new_title = True
        feature.append(title)
    tfidf = text.TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(feature)
    similarity = cosine_similarity(tfidf_matrix)
    del tfidf
    del tfidf_matrix
    if (new_title):
        del feature
        return similarity[len(similarity) - 1]
    else:
        del feature
        index = pd.Series(indices[title])
        return similarity[index[0]]

In [7]:
# Generate Recommendation
def Anime_Recommendation(ID, title, genre):
    cleaned_title = clean(title)
    cleaned_genre = clean(genre)
    cleaned_TitleGenre = cleaned_title + " " + cleaned_genre
    similarity = get_similarity(cleaned_TitleGenre)
    similarity_scores = list(enumerate(similarity))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = filter(lambda x: x[1] > 0, similarity_scores)
    similarity_scores = list(similarity_scores)
    movieindices = [i[0] for i in similarity_scores if i[0] < len(data)]
    scores = [i[1] for i in similarity_scores if i[0] < len(data)]
    result = pd.DataFrame([data.iloc[i] for i in movieindices])
    result['Similarity'] = scores
    result = result[result["ID"] != ID]
    result = result[['Title', 'Similarity']][:10].copy()
    result = result.set_index([np.arange(1,11)])
    print("Title:", title)
    print("Genre:", genre)
    print("Anime Recommendation:")
    print(tabulate(result, headers='keys', tablefmt = 'psql'))
            
    del similarity
    del similarity_scores
    del movieindices
    del scores
    del result

In [8]:
def Anime(ID):
    anime = data.loc[data["ID"] == ID]
    title = anime.Title.item()
    genre = anime.Genre.item()
    Anime_Recommendation(ID, title, genre)

Anime(20)

Title: Naruto
Genre: Action, Adventure, Comedy, Super Power, Martial Arts, Shounen
Anime Recommendation:
+----+----------------------------------------------------------+--------------+
|    | Title                                                    |   Similarity |
|----+----------------------------------------------------------+--------------|
|  1 | Naruto: Shippuuden                                       |     0.834171 |
|  2 | Naruto x UT                                              |     0.760086 |
|  3 | Boruto: Naruto the Movie                                 |     0.741371 |
|  4 | The Last: Naruto the Movie                               |     0.686851 |
|  5 | Naruto: Shippuuden Movie 6 - Road to Ninja               |     0.648502 |
|  6 | Boruto: Naruto Next Generations                          |     0.640321 |
|  7 | Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi  |     0.630476 |
|  8 | Naruto: Honoo no Chuunin Shiken! Naruto vs. Konohamaru!! |     0.626463 |
|  9

In [9]:
Anime(34566)

Title: Boruto: Naruto Next Generations
Genre: Action, Adventure, Super Power, Martial Arts, Shounen
Anime Recommendation:
+----+----------------------------------------------------------+--------------+
|    | Title                                                    |   Similarity |
|----+----------------------------------------------------------+--------------|
|  1 | Boruto: Naruto the Movie                                 |     0.725436 |
|  2 | Kamiusagi Rope x Boruto: Naruto Next Generations         |     0.692526 |
|  3 | Naruto                                                   |     0.640321 |
|  4 | Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi  |     0.564011 |
|  5 | Naruto: Shippuuden                                       |     0.534137 |
|  6 | Boruto: Jump Festa 2016 Special                          |     0.507336 |
|  7 | Naruto x UT                                              |     0.486056 |
|  8 | The Last: Naruto the Movie                               |   

In [10]:
Anime(30694)

Title: Dragon Ball Super
Genre: Action, Adventure, Comedy, Super Power, Martial Arts, Fantasy, Shounen
Anime Recommendation:
+----+-----------------------------------------+--------------+
|    | Title                                   |   Similarity |
|----+-----------------------------------------+--------------|
|  1 | Dragon Ball Z                           |     0.967744 |
|  2 | Dragon Ball                             |     0.955145 |
|  3 | Super Dragon Ball Heroes                |     0.874004 |
|  4 | Dragon Ball Kai                         |     0.857206 |
|  5 | Dragon Ball Kai (2014)                  |     0.857206 |
|  6 | Dragon Ball Super Movie: Broly          |     0.826302 |
|  7 | Dragon Ball Z Movie 15: Fukkatsu no "F" |     0.800821 |
|  8 | Dragon Ball Specials                    |     0.781349 |
|  9 | Dragon Ball: Episode of Bardock         |     0.723416 |
| 10 | Dragon Ball Z: Summer Vacation Special  |     0.71557  |
+----+-------------------------------------

If you found our implementation useful, please consider citing our paper:

C. G. Reswara, J. Nicolas, M. Ananta and F. I. Kurniadi, "Anime Recommendation System Using Bert and Cosine Similarity," 2023 4th International Conference on Artificial Intelligence and Data Sciences (AiDAS), IPOH, Malaysia, 2023, pp. 109-113, doi: [10.1109/AiDAS60501.2023.10284693](https://doi.org/10.1109/AiDAS60501.2023.10284693).